In [1]:
import threading
import gym
import tensorflow as tf

In [2]:
import rl

In [5]:
env = gym.make('LunarLander-v2')

[2016-11-06 10:38:34,397] Making new env: LunarLander-v2


In [6]:
env.observation_space

Box(8,)

In [7]:
env.action_space

Discrete(4)

In [8]:
def Networks(env, states, hidden_num):
    """Maps states tensor to the pair of policy and value tensors."""
    with tf.device("/cpu:0"):
        with tf.variable_scope('common'):
            hidden = tf.contrib.layers.relu(states, hidden_num,
                                            weights_initializer=tf.truncated_normal_initializer(stddev=1.),
                                            biases_initializer=tf.constant_initializer(0.1))
        with tf.variable_scope('policy'):
            logits = tf.contrib.layers.linear(hidden, env.action_space.n)
        with tf.variable_scope('value'):
            value_out = tf.contrib.layers.linear(hidden, 1)
        return logits, value_out

In [9]:
options = dict(rl.DEFAULT_OPTIONS)
options['threads'] = 4
options['learning_rate'] = 0.0001

In [10]:
ac = rl.ActorCritic(lambda: gym.make('LunarLander-v2'),
                    lambda env, st: Networks(env, st, 20),
                    options=options)

[2016-11-06 10:39:05,728] Making new env: LunarLander-v2


In [11]:
logs_dir = '/media/vertix/UHDD/tmp/tensorflow_logs/{}/{:02d}'.format(env.spec.id, 1)
ac.Init(logs_dir)

In [12]:
ac.Learn(10000)

[2016-11-06 10:39:13,827] Making new env: LunarLander-v2
[2016-11-06 10:39:13,829] Making new env: LunarLander-v2
[2016-11-06 10:39:13,829] Making new env: LunarLander-v2
[2016-11-06 10:39:13,831] Making new env: LunarLander-v2


In [12]:
try:
    done = False
    obs = env.reset()
    r = 0
    while not done:
        env.render()
        a = ac.CleanPolicy(obs)
        obs, reward, done, _ = env.step(a)
        r += reward
    print r
finally:
    env.render(close=True)

-506.566992305


In [11]:
ac.Close()